In [1]:
import pandas as pd
import numpy as np
import unidecode
import Levenshtein

In [2]:
df_tratado = pd.read_excel('../../dados_finais_2024/final_data.xlsx', sheet_name='Súmulas')
df_nomes = pd.read_csv('../../scrapers/cleaned_data_2024/final_data_regata.csv', low_memory=False)

In [3]:
df_tratado.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição', 'Regata'],
      dtype='object')

In [4]:
df_tratado.head(2)

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
0,1,1,"WILLIAM JONES, EVAN DEPAUL",17.0,49ER,11,1,GERAL,1,UFD,44.0,33.0,World Championship 2017,NaN
1,2,2,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7.0,49ER,3,0,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022,NaN


In [5]:
df_nomes.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição', 'Regata', 'Nacionalidade'],
      dtype='object')

In [6]:
df_nomes.head(2)

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata,Nacionalidade
0,1,3502,MINA MOBEKK,19,IQFOIL FEM.,15.0,0,NaN,1,0,207,116,European Championship 2023,1Q,NOR
1,2,3502,MINA MOBEKK,19,IQFOIL FEM.,2.0,0,NaN,1,0,207,116,European Championship 2023,2Q,NOR


In [7]:
df_nomes['Nome Competidor'] = df_nomes['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())
df_tratado['Nome Competidor'] = df_tratado['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())

In [8]:
competicoes_tratadas = df_tratado['Nome Competição'].unique()
competicoes_sujas = df_nomes['Nome Competição'].unique()
for competicao in competicoes_sujas:
    if competicao not in competicoes_tratadas:
        print(competicao)

European Championship 2021 - Gold Fleet & Medal Race
European Championship 2021 - Silver Fleet
European Championship 2023 - Bronze Fleet
European Championship 2023 - Gold Fleet
European Championship 2023 - Silver Fleet


In [9]:
for competicao in df_nomes['Nome Competição'].unique():
    if competicao not in df_tratado['Nome Competição'].unique():
        df_nomes[df_nomes['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()

C:\Users\silla\AppData\Local\Temp\ipykernel_4504\1998865097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'European Championship 2021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_nomes[df_nomes['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()
C:\Users\silla\AppData\Local\Temp\ipykernel_4504\1998865097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'European Championship 2021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_nomes[df_nomes['Nome Competição'] == competicao] = str(competicao).split(' - ')[0].strip()
C:\Users\silla\AppData\Local\Temp\ipykernel_4504\1998865097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. 

In [10]:
competicoes_tratadas = df_tratado['Nome Competição'].unique()
competicoes_sujas = df_nomes['Nome Competição'].unique()
for competicao in competicoes_sujas:
    if competicao not in competicoes_tratadas:
        print(competicao)

In [11]:
nomes_tratados = pd.DataFrame()
nomes_tratados['Concat'] = pd.unique(df_tratado['Nome Competidor'] + " % " + df_tratado['Nome Competição'])
nomes_tratados['Nome Competidor'] = nomes_tratados['Concat'].apply(lambda x: x.split('%')[0].strip())
nomes_tratados['Nome Competição'] = nomes_tratados['Concat'].apply(lambda x: x.split('%')[1].strip())
nomes_tratados.drop(columns=['Concat'], inplace=True)

In [12]:
nomes_sujos = pd.DataFrame()
nomes_sujos['Concat'] = pd.unique(df_nomes['Nome Competidor'] + " % " + df_nomes['Nome Competição'])
nomes_sujos['Nome Competidor'] = nomes_sujos['Concat'].apply(lambda x: x.split('%')[0].strip())
nomes_sujos['Nome Competição'] = nomes_sujos['Concat'].apply(lambda x: x.split('%')[1].strip())
nomes_sujos.drop(columns=['Concat'], inplace=True)

In [24]:
nomes_sujos['Match'] = ''
nomes_sujos['Score'] = 1000
for i, sujo in nomes_sujos.iterrows():
    competicao = sujo['Nome Competição']
    nome = sujo['Nome Competidor']
    nomes_1 = nome.split(' ')

    best_score = 1000
    best_match = nome

    for name_ in nomes_tratados[nomes_tratados['Nome Competição'] == competicao]['Nome Competidor']:
        names_2 = name_.split(' ')
        score = 0

        for n1 in nomes_1:
            best = 1000
            for n2 in names_2:
                score_ = Levenshtein.distance(n1, n2)
                if score_ < best:
                    best = score_
            score += best

        if score < best_score:
            best_score = score
            best_match = name_
    
    nomes_sujos.at[i, 'Match'] = best_match
    nomes_sujos.at[i, 'Score'] = best_score

In [32]:
nomes_sujos[nomes_sujos['Score'] > 6]

,Nome Competidor,Nome Competição,Match,Score
352,PAUL LABORDI RE,Semaine Olympique Francaise De Voile 2022,PAUL LABORDERE,7
430,ZOFIA NOCETI-KLEPACKA,Semaine Olympique Francaise De Voile 2022,ZOFIA NOCETI KLEPACKA,7
489,OLEKSANDR MENDELENKO,Semaine Olympique Francaise De Voile 2022,"WIEBRAND DE VOS, ALEKSANDR MOLIAKOV",8
496,"BUKSAK, DOMINIK / KSAWZISE RAZWBIFICSK IG,DS.Z...",Semaine Olympique Francaise De Voile 2022,"DOMINIK BUKSAK, SZYMON WIERZBICKI",30
497,"OSPOZNAN\r\nSANGONEWS, E,N EMVAINXIMILIANO",Semaine Olympique Francaise De Voile 2022,"MAXIMILIANO AGNESE, NEVIN SNOW",22
...,...,...,...,...
8106,ADRIANA\r\n PENRUDDOCKE,World Championship 2020,ADRIANA PENRUDDOCKE,9
8116,KAMOLWAN\r\n CHANYIM,World Championship 2020,KAMOLWAN CHANYIM,9
8118,PATRICIA\r\n RAULINO,World Championship 2020,"PATRICIA SUAREZ, NICOLE VAN DER VELDEN",10
8124,JEAN BAPTISTE BERNAZ,World Championship 2020,JEAN-BAPTISTE BERNAZ,9
